In [1]:
data_test = spark.read.csv('/FileStore/tables/test.csv',header=True, inferSchema=True)
data_train = spark.read.csv('/FileStore/tables/train.csv', header=True, inferSchema=True)

In [2]:
print data_test.count()
print data_train.count()

In [3]:
data_train = data_train.withColumnRenamed("loss","lable")

In [4]:
[trainingdata, validationdata] = data_train.randomSplit([0.7, 0.3])

In [5]:
trainingdata.cache()
validationdata.cache()


In [6]:
columns = data_train.columns

In [7]:
cat_cols = list(filter(lambda s: s.startswith("cat"), columns))
cont_cols = list(filter(lambda s: s.startswith("cont"), columns))

In [8]:
startIndexList = []
import pyspark.ml.feature as ft

In [9]:
for cols in cat_cols:
  startIndexList.append(ft.StringIndexer(inputCol=cols, outputCol="idx"+cols[3:]))

idx_cat_cols = list(map(lambda s: s.replace("cat", 'idx'), cat_cols))
  

In [10]:
from pyspark.ml import Pipeline, PipelineModel
assembler = ft.VectorAssembler(inputCols = idx_cat_cols+cont_cols, outputCol='features')

In [11]:
from pyspark.ml.regression import RandomForestRegressor
estimator = RandomForestRegressor(numTrees=3,maxDepth=5, maxBins=1100, featuresCol="features",labelCol="label")

In [12]:
from pyspark.ml.regression import RandomForestRegressor

In [13]:
stages = []
stages.extend(startIndexList)
stages.append(assembler)
stages.append(estimator) 



In [14]:
pipeline = Pipeline(stages=stages)

In [15]:
model = pipeline.fit(trainingdata)

In [16]:
model.transform()